In [8]:
from cmj.loa import models
from cmj.utils import Manutencao, run_sql
from django.conf import settings
from django.db.models import Q, F
from django.db.models.aggregates import Sum
from decimal import Decimal
import fitz
import re
m = Manutencao()
m.desativa_auto_now()
m.desativa_signals()

In [9]:
loa = models.Loa.objects.get(pk=2023)

In [10]:
(models.Despesa.objects.filter(loa=loa).delete(), 
models.Orgao.objects.filter(loa=loa).delete(),
models.UnidadeOrcamentaria.objects.filter(loa=loa).delete(),
models.Funcao.objects.filter(loa=loa).delete(),
models.SubFuncao.objects.filter(loa=loa).delete(),
models.Programa.objects.filter(loa=loa).delete(),
models.Acao.objects.filter(loa=loa).delete(),
models.Natureza.objects.filter(loa=loa).delete(),
models.EmendaLoaRegistroContabil.objects.filter(emendaloa__loa=loa).delete())

((0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {'loa.EmendaLoaRegistroContabil': 0}))

In [11]:
d = DocumentoAcessorio.objects.get(pk=2733)
settings.SITE_URL+d.arquivo.url

'https://www.jatai.go.leg.br/api/materia/documentoacessorio/2733/arquivo/'

In [12]:
dfile = fitz.open(d.arquivo.path)
#dfile = fitz.open('/home/leandro/Downloads/anexo_materia.pdf')

In [13]:
def load_pages_with(page_mask=''):
    rtext = ''
    for np in range(len(dfile)):
        page = dfile.load_page(np)
        text = page.get_text("text")

        if page_mask not in text:
            continue

        rtext += text
    while '  ' in rtext:
        rtext = rtext.replace('  ', ' ')
    while ' \n' in rtext:
        rtext = rtext.replace(' \n', '\n')
    while '\n\n' in rtext:
        rtext = rtext.replace('\n\n', '\n')
    return rtext

In [14]:
text_global = load_pages_with(page_mask='Quadro de Detalhamento da Despesa\nEXERCÍCIO DE 2023\n')

In [16]:
len(text_global), text_global[0:1000]

(62815,
 'ESTADO DE GOIÁS\nMUNICÍPIO DE JATAI\nQuadro de Detalhamento da Despesa\nEXERCÍCIO DE 2023\nRUA ITARUMA, 355 - S. SANTA MARIA\n1\nPÁGINA\nUNIDADE ORÇAMENTÁRIA\nÓRGÃO\nCÓDIGO\nCÓDIGO\nAPLICAÇÃO PROGRAMADA\nCÓDIGOS\nDETALHADA\nAPLICADO\nNATUREZA DA\nCLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\nCAMARA MUNICIPAL DE JATAI\nCAMARA MUNICIPAL\n01\n01\nOBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL\n01.031.0139.1.001\n4.4.90.51.00\n448.196,31\n4.4.90.52.00\n403.900,00\n852.096,31\nMANUT.ATIV.DA CAMARA MUNICIPAL\n01.031.0139.2.001\n3.1.90.11.00\n13.500.000,00\n3.1.90.16.00\n2.100,00\n3.1.90.96.00\n60.000,00\n3.3.90.14.00\n70.000,00\n3.3.90.30.00\n630.000,00\n3.3.90.33.00\n501,76\n3.3.90.34.00\n1.788,04\n3.3.90.35.00\n433,98\n3.3.90.36.00\n90.000,00\n3.3.90.37.00\n9.486,80\n3.3.90.39.00\n730.000,00\n3.3.90.40.00\n150.000,00\n3.3.90.92.00\n100,00\n15.244.410,58\nENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA\n01.271.2839.9.061\n3.1.90.13.00\n2.000.000,00\n3.1.90.92.00\n100,00\n2.000.100,00\nENC.PR

In [17]:
aplicacoes = []
for m in re.finditer(r'(.+)\n(\d{2}\.\d{3}\.\d{4}\.\d\.\d{3})', text_global):
    aplicacoes.append([m.start(), m.end(), list(m.groups())])
aplicacoes.append([len(text_global),len(text_global),None])
for i in range(len(aplicacoes)-1):
    aplic = aplicacoes[i]
    x1 = aplicacoes[i][1]
    x2 = aplicacoes[i+1][0]

    natdespesas = []

    for n in re.finditer(r'(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(\d{0,3}(?:\.\d{3})*,\d{2})', text_global[x1:x2]):
        natdespesas.append(list(n.groups()))

    aplic.append(natdespesas)
    
aplicacoes.pop()
aplicacoes = list(map(lambda x: (x[2][1], x[2][0], x[0], x[1], tuple(x[3])), aplicacoes))

In [18]:
aplicacoes

[('01.031.0139.1.001',
  'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
  329,
  382,
  (['4.4.90.51.00', '448.196,31'], ['4.4.90.52.00', '403.900,00'])),
 ('01.031.0139.2.001',
  'MANUT.ATIV.DA CAMARA MUNICIPAL',
  442,
  490,
  (['3.1.90.11.00', '13.500.000,00'],
   ['3.1.90.16.00', '2.100,00'],
   ['3.1.90.96.00', '60.000,00'],
   ['3.3.90.14.00', '70.000,00'],
   ['3.3.90.30.00', '630.000,00'],
   ['3.3.90.33.00', '501,76'],
   ['3.3.90.34.00', '1.788,04'],
   ['3.3.90.35.00', '433,98'],
   ['3.3.90.36.00', '90.000,00'],
   ['3.3.90.37.00', '9.486,80'],
   ['3.3.90.39.00', '730.000,00'],
   ['3.3.90.40.00', '150.000,00'],
   ['3.3.90.92.00', '100,00'])),
 ('01.271.2839.9.061',
  'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
  799,
  857,
  (['3.1.90.13.00', '2.000.000,00'], ['3.1.90.92.00', '100,00'])),
 ('01.272.2839.9.001',
  'ENC.PREV. C/O REGIME PRÓPRIO DE PREVIDÊNCIA',
  917,
  978,
  (['3.1.91.13.00', '1.500.000,00'],
   ['3.1.91.92.00', '100,00'],
   ['3.3.90.08.00', '100,00'],
   ['3

In [19]:
unidades_temp = []
for n in re.finditer(r'(?:CLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\n)(.+)\n(.+)\n(.+)\n(.+)\n', text_global):
    unidades_temp.append([n.start(), n.end(), list(n.groups())])
    
unidades_temp.append([len(text_global),len(text_global),None])

In [20]:
unidades_temp[0]

[231, 329, ['CAMARA MUNICIPAL DE JATAI', 'CAMARA MUNICIPAL', '01', '01']]

In [21]:
unidades = []
for i in range(len(unidades_temp)-1):
    unidade = unidades_temp[i]
    x1 = unidades_temp[i][1]
    x2 = unidades_temp[i+1][0]
    
    unidade = [(unidade[2][2], unidade[2][0]), (unidade[2][3], unidade[2][1]), []]
    for aplic in aplicacoes:
        if aplic[2] >= x1 and aplic[3] <= x2:
            unidade[-1].append(aplic)    
    unidades.append(unidade)    

In [22]:
unidades_temp = unidades
unidades = []

orgao = ''
unidade_orc = ''
for u in unidades_temp:
    if u[0] != orgao or u[1] != unidade_orc:
        orgao = u[0]
        unidade_orc = u[1]
        unidades.append([orgao, unidade_orc, []])
    unidades[-1][-1].extend(u[2])

In [23]:
unidades

[[('01', 'CAMARA MUNICIPAL DE JATAI'),
  ('01', 'CAMARA MUNICIPAL'),
  [('01.031.0139.1.001',
    'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
    329,
    382,
    (['4.4.90.51.00', '448.196,31'], ['4.4.90.52.00', '403.900,00'])),
   ('01.031.0139.2.001',
    'MANUT.ATIV.DA CAMARA MUNICIPAL',
    442,
    490,
    (['3.1.90.11.00', '13.500.000,00'],
     ['3.1.90.16.00', '2.100,00'],
     ['3.1.90.96.00', '60.000,00'],
     ['3.3.90.14.00', '70.000,00'],
     ['3.3.90.30.00', '630.000,00'],
     ['3.3.90.33.00', '501,76'],
     ['3.3.90.34.00', '1.788,04'],
     ['3.3.90.35.00', '433,98'],
     ['3.3.90.36.00', '90.000,00'],
     ['3.3.90.37.00', '9.486,80'],
     ['3.3.90.39.00', '730.000,00'],
     ['3.3.90.40.00', '150.000,00'],
     ['3.3.90.92.00', '100,00'])),
   ('01.271.2839.9.061',
    'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
    799,
    857,
    (['3.1.90.13.00', '2.000.000,00'], ['3.1.90.92.00', '100,00'])),
   ('01.272.2839.9.001',
    'ENC.PREV. C/O REGIME PRÓPRIO DE PREVID

In [24]:
funcoes = re.split(r'\||\n', '''01|LEGISLATIVA
02|JUDICIÁRIA
03|ESSENCIAL A JUSTIÇA
04|ADMINISTRAÇÃO
05|DEFESA NACIONAL
06|SEGURANÇA PÚBLICA
07|RELAÇÕES EXTERIORES
08|ASSISTÊNCIA SOCIAL
09|PREVIDÊNCIA SOCIAL
10|SAÚDE
11|TRABALHO
12|EDUCAÇÃO
13|CULTURA
14|DIREITOS DA CIDADANIA
15|URBANISMO
16|HABITAÇÃO
17|SANEAMENTO
18|GESTÃO AMBIENTAL
19|CIÊNCIAS E TECNOLOGIA
20|AGRICULTURA
21|ORGANIZAÇÃO AGRÁRIA
22|INDÚSTRIA
23|COMÉRCIO E SERVIÇOS
24|COMUNICAÇÕES
25|ENERGIA
26|TRANSPORTE
27|DESPORTO E LAZER
28|ENCARGOS ESPECIAIS
99|RESERVA DE CONTINGÊNCIA''')

In [25]:
funcoes = [(funcoes[i], funcoes[i+1]) for i in range(0, len(funcoes)-1, 2)]

In [26]:
funcoes = sorted(funcoes, key=lambda x: x[0])

In [27]:
for c, e in funcoes:
    created, f = models.Funcao.objects.get_or_create(loa=loa, codigo=c, especificacao=e)

In [28]:
models.Funcao.objects.filter(loa=loa).count()

29

In [29]:
subfs = re.split(r'\t|\n', '''01	031	AÇÃO LEGISLATIVA
01	032	CONTROLE EXTERNO
02	061	AÇÃO JUDICIÁRIA
02	062	DEFESA DO INTERESSE PÚBLICO NO PROCESSO JUDICIÁRIO
03	091	DEFESA DA ORDEM JURÍDICA
03	092	REPRESENTAÇÃO JUDICIAL E EXTRAJUDICIAL
04	121	PLANEJAMENTO E ORÇAMENTO
04	122	ADMINISTRAÇÃO GERAL
04	123	ADMINISTRAÇÃO FINANCEIRA
04	124	CONTROLE INTERNO
04	125	NORMATIZAÇÃO E FISCALIZAÇÃO
04	126	TECNOLOGIA DA INFORMAÇÃO
04	127	ORDENAMENTO TERRITORIAL
04	128	FORMAÇÃO DE RECURSOS HUMANOS
04	129	ADMINISTRAÇÃO DE RECEITAS
04	130	ADMINISTRAÇÃO DE CONCESSÕES
04	131	COMUNICAÇÃO SOCIAL
05	151	DEFESA AÉREA
05	152	DEFESA NAVAL
05	153	DEFESA TERRESTRE
06	181	POLICIAMENTO
06	182	DEFESA CIVIL
06	183	INFORMAÇÃO E INTELIGÊNCIA
07	211	RELAÇÕES DIPLOMÁTICAS
07	212	COOPERAÇÃO INTERNACIONAL
08	241	ASSISTÊNCIA AO IDOSO
08	242	ASSISTÊNCIA AO PORTADOR DE DEFICIÊNCIA
08	243	ASSISTÊNCIA A CRIANÇA E  AO ADOLESCENTE
08	244	ASSISTÊNCIA COMUNITÁRIA
09	271	PREVIDÊNCIA BÁSICA
09	272	PREVIDÊNCIA DO REGIME ESTATUTÁRIO
09	273	PREVIDÊNCIA COMPLEMENTAR
09	274	PREVIDÊNCIA ESPECIAL
10	301	ATENÇÃO BÁSICA
10	302	ASSISTÊNCIA HOSPITALAR E AMBULATORIAL
10	303	SUPORTE PROFILÁTICO E TERAPÊUTICO
10	304	VIGILÂNCIA SANITÁRIA
10	305	VIGILÂNCIA EPIDEMIOLÓGICA
10	306	ALIMENTAÇÃO E NUTRIÇÃO
11	331	PROTEÇÃO E BENEFÍCIOS AO TRABALHADOR
11	332	RELAÇÕES DE TRABALHO
11	333	EMPREGABILIDADE
11	334	FOMENTO AO TRABALHO
12	361	ENSINO FUNDAMENTAL
12	362	ENSINO MÉDIO
12	363	ENSINO PROFISSIONAL
12	364	ENSINO SUPERIOR
12	365	EDUCAÇÃO INFANTIL
12	366	EDUCAÇÃO DE JOVENS E ADULTOS
12	367	EDUCAÇÃO ESPECIAL
12	368	EDUCAÇÃO BÁSICA
13	391	PATRIMÔNIO HISTÓRICO, ARTÍSTICO E ARQUEOLÓGICO
13	392	DIFUSÃO CULTURAL
14	421	CUSTÓDIA E REINTEGRAÇÃO SOCIAL
14	422	DIREITOS INDIVIDUAIS, COLETIVOS E DIFUSOS
14	423	ASSISTÊNCIA AOS POVOS INDÍGENAS
15	451	INFRA-ESTRUTURA URBANA
15	452	SERVIÇOS URBANOS
15	453	TRANSPORTES COLETIVOS URBANOS
16	481	HABITAÇÃO RURAL
16	482	HABITAÇÃO URBANA
17	511	SANEAMENTO BÁSICO RURAL
17	512	SANEAMENTO BÁSICO URBANO
18	541	PRESERVAÇÃO E CONSERVAÇÃO AMBIENTAL
18	542	CONTROLE AMBIENTAL
18	543	RECUPERAÇÃO DE ÁREAS DEGRADADAS
18	544	RECURSOS HÍDRICOS
18	545	METEOROLOGIA
19	571	DESENVOLVIMENTO CIENTÍFICO
19	572	DESENVOLVIMENTO TECNOLÓGICO E ENGENHARIA
19	573	DIFUSÃO DO CONHECIMENTO CIENTÍFICO E TECNOLÓGICO
20	601	PROMOÇÃO DA PRODUÇÃO VEGETAL
20	602	PROMOÇÃO DA PRODUÇÃO ANIMAL
20	603	DEFESA SANITÁRIA VEGETAL
20	604	DEFESA SANITÁRIA ANIMAL
20	605	ABASTECIMENTO
20	606	EXTENSÃO RURAL
20	607	IRRIGAÇÃO
20	608	PROMOÇÃO DA PRODUÇÃO AGROPECUARIA
20	609	DEFESA AGROPECAUARIA
21	631	REFORMA AGRÁRIA
21	632	COLONIZAÇÃO
22	661	PROMOÇÃO INDUSTRIAL
22	662	PRODUÇÃO INDUSTRIAL
22	663	MINERAÇÃO
22	664	PROPRIEDADE INDUSTRIAL
22	665	NORMALIZAÇÃO E QUALIDADE
23	691	PROMOÇÃO COMERCIAL
23	692	COMERCIALIZAÇÃO
23	693	COMÉRCIO EXTERIOR
23	694	SERVIÇOS FINANCEIROS
23	695	TURISMO
24	721	COMUNICAÇÕES POSTAIS
24	722	TELECOMUNICAÇÕES
25	751	CONSERVAÇÃO DE ENERGIA
25	752	ENERGIA ELÉTRICA
25	753	COMBUSTÍVEIS MINERAIS
25	754	BIOCOMBUSTÍVEIS
26	781	TRANSPORTE AÉREO
26	782	TRANSPORTE RODOVIÁRIO
26	783	TRANSPORTE FERROVIÁRIO
26	784	TRANSPORTE HIDROVIÁRIO
26	785	TRANSPORTE ESPECIAIS
27	811	DESPORTO DE RENDIMENTO
27	812	DESPORTO COMUNITÁRIO
27	813	LAZER
28	841	REFINANCIAMENTO DA DÍVIDA INTERNA
28	842	REFINANCIAMENTO DA DÍVIDA EXTERNA
28	843	SERVIÇO DA DÍVIDA INTERNA
28	844	SERVIÇO DA DÍVIDA EXTERNA
28	845	OUTRAS TRANSFERÊNCIAS
28	846	OUTROS ENCARGOS ESPECIAIS
28	847	TRANSFERÊNCIAS PARA A EDUCAÇÃO BÁSICA
99	999	RESERVA DE CONTINGÊNCIA''')

In [30]:
subfs = [(subfs[i], subfs[i+1], subfs[i+2]) for i in range(0, len(subfs)-2, 3)]

In [31]:
subfs = sorted(subfs, key=lambda x: (x[0], x[1]))

In [32]:
for fc, c, e in subfs:
    f = models.Funcao.objects.filter(loa=loa, codigo=fc).first()
    created, sf = models.SubFuncao.objects.get_or_create(loa=loa, funcao=f, codigo=c, especificacao=e)

In [33]:
models.SubFuncao.objects.filter(loa=loa).count()

114

In [34]:
count = [0,0,0]
for orgao, unidade, despesas in unidades:
    count[0] += 1
    for desp, espec, x1, x2, naturezas in despesas:
        count[1] += 1
        for nat in naturezas:
            count[2] += 1
print(count)

[31, 241, 1026]


In [35]:
count = 0
for orgao, unidade, despesas in unidades:
    o_obj, created = models.Orgao.objects.get_or_create(
        loa=loa, codigo=orgao[0], especificacao=orgao[1]
    )

    uo_obj, created = models.UnidadeOrcamentaria.objects.get_or_create(
        loa=loa, orgao=o_obj, codigo=unidade[0], especificacao=unidade[1]
    )
    
    for desp, espec, x1, x2, naturezas in despesas:
        desp_re = re.match(r'(?P<funcao>\d{2})\.(?P<subfuncao>\d{3})'
                           r'\.(?P<programa>\d{4})\.(?P<acao>\d\.\d{3})', desp)

        funcao_obj = models.Funcao.objects.filter(loa=loa, codigo=desp_re['funcao']).first()
        subfunc_obj = models.SubFuncao.objects.filter(loa=loa,
            codigo=desp_re['subfuncao']).first()
        
        programa_obj, created = models.Programa.objects.get_or_create(
            loa=loa, codigo=desp_re['programa'])
        
        acao_obj, created = models.Acao.objects.get_or_create(
            loa=loa, codigo=desp_re['acao'], especificacao=espec)
        
        for nat in naturezas:
            count += 1
            natureza, created = models.Natureza.objects.get_or_create(
            loa=loa, codigo=nat[0])
            
            desp_obj, created = models.Despesa.objects.get_or_create(
            loa = loa,
            orgao = o_obj,
            unidade = uo_obj,
            funcao = funcao_obj,
            subfuncao = subfunc_obj,
            programa = programa_obj,
            acao = acao_obj,
            natureza = natureza)
            
            desp_obj.valor_materia = Decimal(nat[1].replace('.', '').replace(',', '.'))
            
            try: 
                desp_obj.save()
            except Exception as e:
                print(e)
                break
        if not desp_obj.pk:
            break
print(count, models.Despesa.objects.filter(loa=loa).count())

1026 1026


In [36]:
a = """text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.15 DE 20/06/78')
naturezas = []
for m in re.finditer(r'\n?(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n([^0-9]*)?\n?', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])
for esp, cod in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()
    """

In [37]:
text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.08 DE 04/02/85')
naturezas = []
for m in re.finditer(r'\n(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(.*)\n', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])

In [39]:
text_global[0:1000]

'ESTADO DE GOIÁS\nMUNICÍPIO DE JATAI\nLei 4.320/64 - Anexo II\nADENDO III A PORTARIA SOF N.08 DE 04/02/85\nEXERCÍCIO DE 2023\nRUA ITARUMA, 355 - S. SANTA MARIA\nPÁGINA\nORCAMENTO PROGRAMA - ADMINISTRACAO DIRETA E INDIRETA\n1\nCÓDIGO\nESPECIFICAÇÕES\nELEMENTO\nNATUREZA\nCAT.ECONÔMICA\n3.0.00.00.00\nDESPESAS CORRENTES\n18.997.903,69\n3.1.00.00.00\nPESSOAL\n17.315.193,11\n3.1.90.11.00\nVencimentos e Vantagens Fixas - Pessoal Civil\n13.500.000,00\n3.1.90.13.00\nObrigacoes Patronais\n2.000.000,00\n3.1.90.16.00\nOutras Despesas Variaveis - Pessoal Civil\n2.100,00\n3.1.90.91.00\nSentencas Judiciais\n3.574,57\n3.1.90.92.00\nDespesa de Exercício anterior\n100,00\n3.1.90.94.00\nIndenizacoes Restituicoes Trabalhistas\n249.318,54\n3.1.90.96.00\nRessarcimento de Despesas de Pessoal Requisitado\n60.000,00\n3.1.91.13.00\nContribuição Patronal para o RPPS\n1.500.000,00\n3.1.91.92.00\nDespesas de Exercicios Anteriores\n100,00\n3.3.00.00.00\nOUTRAS DESPESAS CORRENTES\n1.682.710,58\n3.3.90.08.00\nOutros 

In [40]:
for cod, esp in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()

In [41]:
text_global = load_pages_with(page_mask='ADENDO V A PORTARIA SOF N.15 DE 20/06/78')

In [42]:
programas = set()
for m in re.finditer(r'\n(\d{4})\n(.+)\n', text_global):
    programas.add(m.groups())

In [43]:
programas

{('0139', 'PROCESSO LEGISLATIVO'),
 ('0239', 'APOIO AO PODER JUDICIÁRIO'),
 ('0439', 'PROGRAMA GESTÃO MODERNA E TRANSFORMADORA'),
 ('0639', 'DEFESA SOCIAL E SEGURANCA PUBLICA PRESENTE'),
 ('0839', 'JATAÍ INCLUSÃO SOCIAL A TODOS'),
 ('0939', 'GESTÃO DA SEGURIDADE SOCIAL DOS SERVIDORES'),
 ('1039', 'ASSISTÊNCIA A SAÚDE'),
 ('1239', 'EDUCAÇÃO EM FOCO'),
 ('1339', 'ARTE E CIDADANIA'),
 ('1539', 'AVANÇO NAS MELHORIAS DOS SERVIÇOS DE DESENVOLVIMENTO URBANO'),
 ('1639', 'FAMÍLIA HABITAÇÃO DIGNA'),
 ('1839', 'POLÍTICAS AMBIENTAIS E A MELHORIA NA QUALIDADE DE VIDA'),
 ('1939', 'PROGRAMA DE APOIO À CIÊNCIA, TECNOLOGIA E INOVAÇÃO'),
 ('2039', 'PROGRAMA DE INFRAESTRUTURA E APOIO TÉCNICO AO MORADOR E PRODUTOR'),
 ('2239', 'PROGRAMA DE DESENVOLVIMENTO ECONÔMICO DE JATAÍ'),
 ('2339', 'TURISMO EM AVANÇO'),
 ('2639', 'TRANSPORTES'),
 ('2739', 'VALORIZAÇÃO DO ESPORTE E LAZER'),
 ('2839', 'ENCARGOS ESPECIAIS'),
 ('7999', 'RESERVA TÉCNICA DO RPPS'),
 ('9999', 'RESERVA DE CONTIGENCIA')}

In [46]:
len(programas), models.Programa.objects.filter(loa=loa).values_list('codigo', flat=True).count()

(21, 21)

In [47]:
for cod, esp in list(programas):
    prog, created = models.Programa.objects.get_or_create(loa=loa, codigo=cod)
    prog.especificacao=esp
    prog.save()

In [48]:
models.Despesa.objects.order_by('-id').all().values()[:2]

<QuerySet [{'id': 5522, 'loa_id': 2023, 'orgao_id': 59, 'unidade_id': 127, 'funcao_id': 91, 'subfuncao_id': 350, 'programa_id': 87, 'acao_id': 1179, 'fonte_id': None, 'natureza_id': 242, 'valor_materia': Decimal('1050.00'), 'valor_norma': Decimal('0.00')}, {'id': 5521, 'loa_id': 2023, 'orgao_id': 59, 'unidade_id': 127, 'funcao_id': 91, 'subfuncao_id': 350, 'programa_id': 87, 'acao_id': 1179, 'fonte_id': None, 'natureza_id': 226, 'valor_materia': Decimal('500.00'), 'valor_norma': Decimal('0.00')}]>

In [49]:
#for desp in models.Despesa.objects.order_by('-id').filter(acao_id=1531):
#    print(desp.orgao.codigo, ' - ', desp.orgao.especificacao, f"'{desp.natureza.codigo}'", desp.valor_norma == desp.valor_materia)

In [50]:
#for desp in models.Despesa.objects.order_by('-id').exclude(valor_materia=F('valor_norma')):
#    print(desp, f"'{desp.natureza.codigo}'", desp.valor_norma, desp.valor_materia)

In [55]:
soma_total = models.Despesa.objects.filter(loa=loa).aggregate(vm=Sum('valor_materia'), vn=Sum('valor_norma'))

In [56]:
soma_total, soma_total['vm'] - soma_total['vn']

({'vm': Decimal('641984728.31'), 'vn': Decimal('0.00')},
 Decimal('641984728.31'))

In [57]:
for soma in models.Despesa.objects.filter(loa=loa).values(
    'orgao__codigo', 'unidade__codigo'
).order_by('orgao__codigo', 'unidade__codigo').annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma')):
    #del soma['orgao__codigo']
    print(soma)

{'orgao__codigo': '01', 'unidade__codigo': '01', 'vm': Decimal('19850000.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '01', 'unidade__codigo': '02', 'vm': Decimal('150000.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '02', 'vm': Decimal('8188900.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '03', 'vm': Decimal('50157350.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '06', 'vm': Decimal('32219071.21'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '08', 'vm': Decimal('62772219.81'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '10', 'vm': Decimal('7025501.45'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '11', 'vm': Decimal('132616300.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '15', 'vm': Decimal('10789000.00'), 'vn': Decimal('0.00')}
{'orgao__codigo': '03', 'unidade__codigo': '16', 'vm': Decimal('6281560.81'), 'vn': D

In [58]:
# mostra unidades que totais mudaram da matéria para a norma
sp = 0
sn = 0
for soma in models.Despesa.objects.filter(loa=loa).values(
    'orgao__codigo', 'unidade__codigo'
).order_by('orgao__codigo', 'unidade__codigo').annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma')):
    if soma['vm'] - soma['vn']:
        del soma['orgao__codigo']
        s = soma['vm'] - soma['vn']
        
        if s > 0:
            sp += s
        else:
            sn += s
    
        print(soma, soma['vm'] - soma['vn'])
    #print(soma['unidade__codigo'], soma['vm'] - soma['vn'])
print(sp, sn) #11.035.815,25

{'unidade__codigo': '01', 'vm': Decimal('19850000.00'), 'vn': Decimal('0.00')} 19850000.00
{'unidade__codigo': '02', 'vm': Decimal('150000.00'), 'vn': Decimal('0.00')} 150000.00
{'unidade__codigo': '02', 'vm': Decimal('8188900.00'), 'vn': Decimal('0.00')} 8188900.00
{'unidade__codigo': '03', 'vm': Decimal('50157350.00'), 'vn': Decimal('0.00')} 50157350.00
{'unidade__codigo': '06', 'vm': Decimal('32219071.21'), 'vn': Decimal('0.00')} 32219071.21
{'unidade__codigo': '08', 'vm': Decimal('62772219.81'), 'vn': Decimal('0.00')} 62772219.81
{'unidade__codigo': '10', 'vm': Decimal('7025501.45'), 'vn': Decimal('0.00')} 7025501.45
{'unidade__codigo': '11', 'vm': Decimal('132616300.00'), 'vn': Decimal('0.00')} 132616300.00
{'unidade__codigo': '15', 'vm': Decimal('10789000.00'), 'vn': Decimal('0.00')} 10789000.00
{'unidade__codigo': '16', 'vm': Decimal('6281560.81'), 'vn': Decimal('0.00')} 6281560.81
{'unidade__codigo': '32', 'vm': Decimal('1682995.97'), 'vn': Decimal('0.00')} 1682995.97
{'unidade

In [59]:
for soma in models.Despesa.objects.filter(loa=loa).values(
    'orgao__codigo', 
).order_by('orgao__codigo', 'unidade__codigo', 'acao__codigo'
          ).annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma')
                    ):
    if soma['vm'] == soma['vn']:
        continue
    del soma['orgao__codigo']
    #del soma['unidade__codigo']
    #rint(soma['acao__codigo'], soma['vm'] - soma['vn'])

In [61]:
#mostra despesas de uma unidade
for soma in models.Despesa.objects.filter(loa=loa, orgao__codigo='03', unidade__codigo='35').values(
    'acao__codigo', 'natureza__codigo', 'valor_materia', 'valor_norma'
):
    del soma['acao__codigo']
    del soma['valor_materia']
    print(soma) #35 - 116.778,08  #36 - 202.657,51 

{'natureza__codigo': '4.4.90.52.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '4.4.90.52.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '4.4.90.51.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.04.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.11.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.91.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.1.90.94.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.08.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.14.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.30.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.32.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.34.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.36.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.39.00', 'valor_norma': Decimal('0.00')}
{'natureza__codigo': '3.3.90.40.00

In [62]:
for soma in models.Despesa.objects.filter(loa=loa, orgao__codigo='01').values(
    'subfuncao__codigo', 'subfuncao__especificacao'
).order_by('subfuncao__codigo').annotate(vm=Sum('valor_materia'), vn=Sum('valor_norma'), alt=Sum('registrocontabil_set__valor')):
    #del soma['subfuncao__codigo']
    print(soma)

{'subfuncao__codigo': '031', 'subfuncao__especificacao': 'AÇÃO LEGISLATIVA', 'vm': Decimal('16246506.89'), 'vn': Decimal('0.00'), 'alt': None}
{'subfuncao__codigo': '271', 'subfuncao__especificacao': 'PREVIDÊNCIA BÁSICA', 'vm': Decimal('2000100.00'), 'vn': Decimal('0.00'), 'alt': None}
{'subfuncao__codigo': '272', 'subfuncao__especificacao': 'PREVIDÊNCIA DO REGIME ESTATUTÁRIO', 'vm': Decimal('1500300.00'), 'vn': Decimal('0.00'), 'alt': None}
{'subfuncao__codigo': '846', 'subfuncao__especificacao': 'OUTROS ENCARGOS ESPECIAIS', 'vm': Decimal('253093.11'), 'vn': Decimal('0.00'), 'alt': None}


In [63]:
for soma in models.Despesa.objects.filter(loa=loa, orgao__codigo='01', unidade__codigo='02'):
    print(soma)

01.031.0139.1.149:4.4.90.52.00 - OBRAS E EQUIPAMENTOS DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.14.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.30.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.36.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO
01.031.0139.2.101:3.3.90.39.00 - MANUTENÇÃO DE ATIVIDADES DA ESCOLA DO LEGISLATIVO


In [64]:
Acao.objects.filter(loa=loa).count()

241

In [66]:
result = run_sql('''SELECT
  SUBSTRING(n.codigo, 1, 1) AS nat,
  
  sum(d.valor_materia)
    from loa_despesa d
        inner join loa_loa        l on (d.loa_id = l.id)
        inner join loa_natureza   n on (d.natureza_id = n.id)
    where
        d.loa_id = 2023
    group by nat
    order by nat

;''')
result

[('3', Decimal('439448598.34')),
 ('4', Decimal('188782356.72')),
 ('7', Decimal('8590255.26')),
 ('9', Decimal('5163517.99'))]

In [67]:
for r in result[:10]:
    print(r)

('3', Decimal('439448598.34'))
('4', Decimal('188782356.72'))
('7', Decimal('8590255.26'))
('9', Decimal('5163517.99'))
